In [1]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder \
    .appName("Footbal project") \
    .getOrCreate()

results_df = spark.read.option("header", "true").csv("results.csv")
goals_df = spark.read.option("header", "true").csv("goalscorers.csv")

results_df.show(5)
goals_df.show(5)


+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  FALSE|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  FALSE|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  FALSE|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  FALSE|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  FALSE|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 5 rows
+----------+---------+---------+---------+----------------+------+--------+-------+
|      date|home_team|away_team|     team|          scorer|minute|own_goa

Prvi zadatak: najbolja i najgora gol razlika Srbije 

In [ ]:
rdd = results_df.rdd.map(lambda row: (
    row['home_team'], 
    row['away_team'], 
    int(row['home_score']), 
    int(row['away_score'])
))

serbia_rdd = rdd.filter(lambda x: x[0] == 'Serbia' or x[1] == 'Serbia')

def map_goal_diff(match):
    home, away, home_score, away_score = match
    if home == 'Serbia':
        opponent = away
        diff = home_score - away_score
    else:
        opponent = home
        diff = away_score - home_score
    return (opponent, diff)

mapped = serbia_rdd.map(map_goal_diff)
reduced = mapped.reduceByKey(lambda a, b: a + b)

best = reduced.max(key=lambda x: x[1])
worst = reduced.min(key=lambda x: x[1])

print("Najbolja gol razlika sa Srbijom:", best)
print("Najgora gol razlika sa Srbijom:", worst)

Najbolja gol razlika:
Najgora gol razlika:

Drugi zadatak: Fudbaleri sa het-trikom na Evropskom

In [7]:
df2 = goals_df.join(results_df, on=["date", "home_team", "away_team"], how="left")

euro_df = df2.filter(F.col("tournament") == "UEFA Euro")

hat_tricks = (
    euro_df.groupBy("date", "home_team", "away_team", "team", "scorer", "tournament", "country")
    .agg(F.count("*").alias("golova"))
    .filter(F.col("golova") >= 3)
)

hat_tricks.select(
    F.col("date").alias("Datum"),
    F.col("home_team").alias("Domaći"),
    F.col("away_team").alias("Gosti"),
    F.col("team").alias("Igra za"),
    F.col("scorer").alias("Strelac"),
    F.col("golova").alias("Broj golova"),
    F.col("tournament").alias("Turnir"),
    F.col("country").alias("Zemlja")
).show(truncate=False)

+----------+-----------+-----------+-----------+----------------+-----------+---------+-----------+
|Datum     |Domaći     |Gosti      |Igra za    |Strelac         |Broj golova|Turnir   |Zemlja     |
+----------+-----------+-----------+-----------+----------------+-----------+---------+-----------+
|1984-06-19|France     |Yugoslavia |France     |Michel Platini  |3          |UEFA Euro|France     |
|1984-06-16|France     |Belgium    |France     |Michel Platini  |3          |UEFA Euro|France     |
|2000-06-25|Netherlands|Serbia     |Netherlands|Patrick Kluivert|3          |UEFA Euro|Netherlands|
|1976-06-17|Yugoslavia |Germany    |Germany    |Dieter Müller   |3          |UEFA Euro|Yugoslavia |
|2000-06-20|Portugal   |Germany    |Portugal   |Sérgio Conceição|3          |UEFA Euro|Netherlands|
|1988-06-15|England    |Netherlands|Netherlands|Marco van Basten|3          |UEFA Euro|Germany    |
|1980-06-14|Germany    |Netherlands|Germany    |Klaus Allofs    |3          |UEFA Euro|Italy      |


+----------+-----------+-----------+-----------+----------------+-----------+---------+-----------+
|Datum     |Domaći     |Gosti      |Igra za    |Strelac         |Broj golova|Turnir   |Zemlja     |
+----------+-----------+-----------+-----------+----------------+-----------+---------+-----------+
|1984-06-19|France     |Yugoslavia |France     |Michel Platini  |3          |UEFA Euro|France     |
|1984-06-16|France     |Belgium    |France     |Michel Platini  |3          |UEFA Euro|France     |
|2000-06-25|Netherlands|Serbia     |Netherlands|Patrick Kluivert|3          |UEFA Euro|Netherlands|
|1976-06-17|Yugoslavia |Germany    |Germany    |Dieter Müller   |3          |UEFA Euro|Yugoslavia |
|2000-06-20|Portugal   |Germany    |Portugal   |Sérgio Conceição|3          |UEFA Euro|Netherlands|
|1988-06-15|England    |Netherlands|Netherlands|Marco van Basten|3          |UEFA Euro|Germany    |
|1980-06-14|Germany    |Netherlands|Germany    |Klaus Allofs    |3          |UEFA Euro|Italy      |
|2008-06-10|Spain      |Russia     |Spain      |David Villa     |3          |UEFA Euro|Austria    |
+----------+-----------+-----------+-----------+----------------+-----------+---------+-----------+
